In [2]:
from dotenv import load_dotenv
import os
load_dotenv(override=True)

True

Using selenium to log in:

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

In [10]:
class DataLoader():
    def __init__(self, url_prefix: str):
        self.driver = webdriver.Chrome()
        self.url_prefix = url_prefix
        self.username = os.environ['USERNAME']
        self.password = os.environ['PASSWORD']
    
    def __enter__(self):
        return self
    
    def __exit__(self, type, value, traceback):
        self.driver.close()
    
    def login(self):
        self.driver.get(f"{self.url_prefix}/login")
        self.driver.find_element(by="id", value="loginName").send_keys(self.username)
        self.driver.find_element(value="submit").click()
        self.driver.find_element(value="password").send_keys(self.password)
        self.driver.find_element(value="submit").click()

    @staticmethod
    def preprocess(text: str) -> str:
        text = text.replace(u'\xa0', u' ')
        text = text.strip()
        return text
    
    def get_html(self, url_suffix: str) -> str:
        self.driver.get(f"{self.url_prefix}/{url_suffix}")
        return self.driver.page_source
    
    def get_page_data(self, url_suffix: str) -> str:
        html = self.get_html(url_suffix)
        soup = BeautifulSoup(html)
        title = soup.find("h1").get_text()
        contents = [paragraph.get_text() for paragraph in soup.find_all("p")]  # TODO: filter out non-content paragraphs
        contents.insert(0, title)
        return "\n\n".join([self.preprocess(text) for text in contents])

In [11]:
session = DataLoader(url_prefix=os.environ["URL_PREFIX"])
session.login()

In [12]:
url_suffix = os.environ["URL_SUFFIX_TEST"]
page = session.get_page_data(url_suffix)